<a href="https://colab.research.google.com/github/bachaudhry/kaggle_birdCLEF_25/blob/main/BirdCLEF25_03_Baseline_Precomputed_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/Kaggle"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q kaggle

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys, gc, random, math, time, copy , zipfile, tarfile, shutil, subprocess, json
from pathlib import Path
from tqdm.notebook import tqdm
import IPython.display as ipd
from IPython.display import display, clear_output
import ipywidgets as widgets

import librosa
import librosa.display
import soundfile as sf

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.multiprocessing as mp
import torchaudio
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader
import torch.amp as amp

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, average_precision_score
from sklearn.preprocessing import LabelEncoder

In [4]:
sys.path.append('/content/drive/MyDrive/Kaggle/Bird_CLEF25/utils')
from utils import Config, BirdClefDataset, create_target_tensor, seed_everything, process_gzipped

In [5]:
cfg = Config()
# Path to original train.csv, audio and metadata
cfg.BASE_DATA_PATH = Path("/content/drive/MyDrive/Kaggle/Bird_CLEF25/data/birdclef-2025")
# Path to npy files
cfg.PRECOMPUTED_SPECS_PATH = Path("/content/drive/MyDrive/Kaggle/Bird_CLEF25/data/precomputed-specs-np-zipped")
# Path to local specs
#cfg.LOCAL_SPECS_PATH = Path("/content/precomputed_spectrograms")
# Training meta data
cfg.TRAIN_METADATA_PATH = Path("/content/drive/MyDrive/Kaggle/Bird_CLEF25/data/birdclef-2025/train.csv")

In [6]:
# --- Set Device & Seed ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
seed_everything(cfg.SEED)

Using device: cpu
Seeded everything with: 42


In [7]:
if cfg.NUM_WORKERS > 0:
    try:
        current_context = mp.get_context(None)
        if not isinstance(current_context, mp.SpawnContext):
             mp.set_start_method('spawn', force=True)
             print("Set multiprocessing start method to 'spawn'.")
        else:
             print("Multiprocessing start method already set to 'spawn'.")
    except RuntimeError as e:
        print(f"Could not set start method (might be already set or first run): {e}")

Set multiprocessing start method to 'spawn'.


In [8]:
# --- Load Metadata ---
if not cfg.TRAIN_METADATA_PATH.exists():
    print(f"ERROR: Metadata file not found at {cfg.TRAIN_METADATA_PATH}")
    # Stop execution or handle
else:
    train_df = pd.read_csv(cfg.TRAIN_METADATA_PATH)
    print(f"Train metadata loaded. Shape: {train_df.shape}")

Train metadata loaded. Shape: (28564, 13)


In [9]:
!pip install gdown
!apt-get install pigz rsync -qq

In [16]:
import gdown
## Testing improved download and unzip function
def process_gzippedV2(input_path, output_path=None):
  local_temp_dir = "/content/temp_data"
  #local_extract_path = os.path.join(local_temp_dir, "extracted")
 # os.makedirs(local_extract_path, exist_ok=True)
  os.makedirs(local_temp_dir, exist_ok=True)

  # Download using gdown - timeout issues
  #print("Downloading compressed file from Google Drive...")
  #url = f'https://drive.google.com/uc?id={file_id.split("/")[-2]}'
  #compressed_path = gdown.download(url, output=local_temp_dir, quiet=False)
  # Ensure compressed_path is a file, not the directory
  #compressed_path = os.path.join(local_temp_dir, compressed_path)  # Corrected line

    # Get filename and local paths
  filename = os.path.basename(input_path)
  local_compressed_path = os.path.join(local_temp_dir, filename)
  local_extract_path = os.path.join(local_temp_dir, 'extracted')
  os.makedirs(local_extract_path, exist_ok=True)

  # Download file from Drive to Colab
  print(f"Copying {filename} from Drive to Colab...")
  drive_path = os.path.join('/content/drive/MyDrive/', input_path)
  shutil.copy2(drive_path, local_compressed_path) # Slower but more robust

  # Extract with parallel decompression (if possible/available)
  print(f"Extracting {os.path.basename(local_compressed_path)}...")
  try:
    # Using pigz for parallel decompression
    subprocess.run(['pigz', '--version'], check=True)
    subprocess.run(['tar', '-I', 'pigz', '-xf', local_compressed_path, '-C', local_extract_path],
                   check=True)
  except:
    # Fallback to tar
    subprocess.run(['tar', '-xzf', local_compressed_path, '-C', local_extract_path],
                   check=True)

  # Clean up compressed file
  os.remove(local_compressed_path)

  # Optional upload to drive
  if output_path:
        print("⏫ Starting Drive upload...")
        drive_output_path = os.path.join('/content/drive/MyDrive', output_path)

        # Use parallel upload with rsync
        subprocess.run([
            'rsync', '-a', '--info=progress2',
            local_extract_path + '/',
            drive_output_path
        ], check=True)

  print("✅ All operations completed!")
  return local_extract_path


In [17]:
#file_id = 'https://drive.google.com/file/d/1Ji5acgpHlyyhd8vI8gyQlN1nkjh16MwN/view?usp=drive_link'
input_path= "/content/drive/MyDrive/Kaggle/Bird_CLEF25/data/precomputed-specs-np-zipped"
local_extract = process_gzippedV2(input_path)

Copying precomputed-specs-np-zipped from Drive to Colab...
Extracting precomputed-specs-np-zipped...
✅ All operations completed!


In [ ]:
# Download zipped folder and extract to local
input_path = "/content/drive/MyDrive/Kaggle/Bird_CLEF25/data/precomputed-specs-np-zipped"
local_extract = process_gzipped(input_path)

Copying precomputed-specs-np-zipped from Drive to Colab...
Extracting precomputed-specs-np-zipped...


Extracting files: 100%|██████████| 187905/187905 [06:03<00:00, 517.40it/s]


Operation completed successfully!


In [18]:
# Update precomputed specs path
#cfg.PRECOMPUTED_SPECS_PATH = cfg.LOCAL_SPECS_PATH
# Run check
local_specs_path = Path("/content/temp_data/extracted/kaggle/working/precomputed_specs_np")
all_precomputed_files = list(local_specs_path.glob("*.npy"))
print(f"Found {len(all_precomputed_files)} precomputed .npy files.")

Found 187904 precomputed .npy files.


In [25]:
all_precomputed_files[1:20]

[PosixPath('/content/temp_data/extracted/kaggle/working/precomputed_specs_np/roahaw_XC113755.ogg_clip1.npy'),
 PosixPath('/content/temp_data/extracted/kaggle/working/precomputed_specs_np/bkcdon_XC250791.ogg_clip2.npy'),
 PosixPath('/content/temp_data/extracted/kaggle/working/precomputed_specs_np/bkmtou1_XC821017.ogg_clip17.npy'),
 PosixPath('/content/temp_data/extracted/kaggle/working/precomputed_specs_np/yebsee1_XC216248.ogg_clip11.npy'),
 PosixPath('/content/temp_data/extracted/kaggle/working/precomputed_specs_np/roahaw_XC925789.ogg_clip16.npy'),
 PosixPath('/content/temp_data/extracted/kaggle/working/precomputed_specs_np/grbhaw1_iNat1156713.ogg_clip0.npy'),
 PosixPath('/content/temp_data/extracted/kaggle/working/precomputed_specs_np/grnkin_XC452703.ogg_clip1.npy'),
 PosixPath('/content/temp_data/extracted/kaggle/working/precomputed_specs_np/paltan1_XC375634.ogg_clip27.npy'),
 PosixPath('/content/temp_data/extracted/kaggle/working/precomputed_specs_np/cotfly1_XC817211.ogg_clip2.npy')

In [60]:
cfg.PRECOMPUTED_SPECS_PATH = local_specs_path

In [19]:
# Create label mappings
unique_labels = sorted(train_df['primary_label'].unique())
cfg.NUM_CLASSES = len(unique_labels)
cfg.LABEL_TO_INT = {label: i for i, label in enumerate(unique_labels)}
cfg.INT_TO_LABEL = {i: label for label, i in cfg.LABEL_TO_INT.items()}
train_df['primary_label_int'] = train_df['primary_label'].map(cfg.LABEL_TO_INT)
print(f"{cfg.NUM_CLASSES} unique classes found.")

206 unique classes found.




---



In [62]:
manifest_path = Path("/content/drive/MyDrive/Kaggle/Bird_CLEF25/data/")

In [73]:
filename_to_label_map = {}

# Efficient lookup using a dictionary from train_df
for _, row in train_df.iterrows():
  original_filename = row['filename']
  # Creating stem as we did for .npy files
  filename_stem_lookup = original_filename.replace('/', '_').replace('.ogg', '')
  filename_to_label_map[filename_stem_lookup] = (row['primary_label_int'], original_filename)

In [74]:
filename_to_label_map

{'1139490_CSA36385': (0, '1139490/CSA36385.ogg'),
 '1139490_CSA36389': (0, '1139490/CSA36389.ogg'),
 '1192948_CSA36358': (1, '1192948/CSA36358.ogg'),
 '1192948_CSA36366': (1, '1192948/CSA36366.ogg'),
 '1192948_CSA36373': (1, '1192948/CSA36373.ogg'),
 '1192948_CSA36388': (1, '1192948/CSA36388.ogg'),
 '1194042_CSA18783': (2, '1194042/CSA18783.ogg'),
 '1194042_CSA18794': (2, '1194042/CSA18794.ogg'),
 '1194042_CSA18802': (2, '1194042/CSA18802.ogg'),
 '126247_XC941297': (3, '126247/XC941297.ogg'),
 '126247_iNat1109254': (3, '126247/iNat1109254.ogg'),
 '126247_iNat146584': (3, '126247/iNat146584.ogg'),
 '126247_iNat320679': (3, '126247/iNat320679.ogg'),
 '126247_iNat888527': (3, '126247/iNat888527.ogg'),
 '126247_iNat888729': (3, '126247/iNat888729.ogg'),
 '1346504_CSA18784': (4, '1346504/CSA18784.ogg'),
 '1346504_CSA18791': (4, '1346504/CSA18791.ogg'),
 '1346504_CSA18792': (4, '1346504/CSA18792.ogg'),
 '1346504_CSA18793': (4, '1346504/CSA18793.ogg'),
 '1346504_CSA18803': (4, '1346504/CSA188



---



In [76]:
print("Sample train_df filenames:")
print(train_df['filename'].head(5).tolist())

print("\nSample .npy filenames from your precomputed directory:")
sample_npy_files = list(cfg.PRECOMPUTED_SPECS_PATH.glob("*.npy"))[:5]
for f_path in sample_npy_files:
    print(f_path.name)

Sample train_df filenames:
['1139490/CSA36385.ogg', '1139490/CSA36389.ogg', '1192948/CSA36358.ogg', '1192948/CSA36366.ogg', '1192948/CSA36373.ogg']

Sample .npy filenames from your precomputed directory:
strowl1_XC48736.ogg_clip1.npy
roahaw_XC113755.ogg_clip1.npy
bkcdon_XC250791.ogg_clip2.npy
bkmtou1_XC821017.ogg_clip17.npy
yebsee1_XC216248.ogg_clip11.npy


In [77]:
# --- CELL 2 REVISED (Attempt 1) ---

# --- Load Metadata ---
if not cfg.TRAIN_METADATA_PATH.exists():
    print(f"ERROR: Metadata file not found at {cfg.TRAIN_METADATA_PATH}")
    raise FileNotFoundError(f"Metadata missing: {cfg.TRAIN_METADATA_PATH}")

train_df = pd.read_csv(cfg.TRAIN_METADATA_PATH)
print(f"Train metadata loaded. Shape: {train_df.shape}")

# --- Create Label Mappings ---
unique_labels = sorted(train_df['primary_label'].unique())
cfg.NUM_CLASSES = len(unique_labels)
cfg.LABEL_TO_INT = {label: i for i, label in enumerate(unique_labels)}
cfg.INT_TO_LABEL = {i: label for label, i in cfg.LABEL_TO_INT.items()}
train_df['primary_label_int'] = train_df['primary_label'].map(cfg.LABEL_TO_INT)
print(f"{cfg.NUM_CLASSES} unique classes found.")

# --- Scan Precomputed Files and Create clip_samples & Manifest ---
all_precomputed_files = list(cfg.PRECOMPUTED_SPECS_PATH.glob("*.npy"))
print(f"Found {len(all_precomputed_files)} precomputed .npy files.")

clip_samples = []
manifest_data = []

if not all_precomputed_files:
    print("ERROR: No precomputed files found.")
else:
    print("Generating clip_info_list and manifest from precomputed files and train_df...")

    # --- Create the lookup dictionary ---
    # Key: The part of the .npy filename BEFORE "_clipIDX"
    # Value: (primary_label_int, original_filename_from_train_df)
    filename_to_label_map = {}
    for _, row in train_df.iterrows():
        original_filename_from_train_df = row['filename']
        # THIS IS THE KEY ASSUMPTION:
        # The key should be the original filename as it appears in train_df,
        # if your .npy files are like "TRAIN_DF_FILENAME_clipIDX.npy"
        # If train_df filenames have '/', and .npy files have them replaced with '_', adjust here.
        # For now, assume .npy naming directly uses train_df['filename'] (potentially with slashes replaced)

        # Let's assume your .npy files look like: "original_filename_from_train_df_WITH_SLASHES_REPLACED_clipIDX.npy"
        # AND that ".ogg" is part of this stem in the .npy file.
        key_for_map = original_filename_from_train_df.replace('/', '_') # Example: "subdir_file.ogg"

        filename_to_label_map[key_for_map] = (row['primary_label_int'], original_filename_from_train_df)

    # --- DEBUG: Print some keys from the map ---
    print(f"Generated {len(filename_to_label_map)} keys for lookup map.")
    print("Sample keys from filename_to_label_map (first 5):")
    for i, key in enumerate(filename_to_label_map.keys()):
        if i < 5:
            print(f"  '{key}' -> {filename_to_label_map[key]}")
        else:
            break
    # --- END DEBUG ---

    processed_count = 0
    for spec_path in tqdm(all_precomputed_files, desc="Mapping precomputed files"):
        try:
            npy_filename_stem_full = spec_path.stem # e.g., "21211_XC934741.ogg_clip0"
            parts = npy_filename_stem_full.rsplit('_clip', 1) # Use rsplit to split on the *last* occurrence

            if len(parts) != 2:
                print(f"Warning: Could not parse clip index from '{spec_path.name}' using '_clip'. Skipping. Full stem: '{npy_filename_stem_full}'")
                continue

            parsed_name_stem_from_npy = parts[0] # This should be the key we look up, e.g., "21211_XC934741.ogg"
                                                # or "ebird_code_XC12345.ogg"

            # --- DEBUG: Print parsed stem from .npy ---
            if processed_count < 5: # Print for the first 5 .npy files
                 print(f"  Attempting to match .npy parsed stem: '{parsed_name_stem_from_npy}'")
            # --- END DEBUG ---

            clip_idx_str = parts[1]
            if not clip_idx_str.isdigit():
                print(f"Warning: Clip index part '{clip_idx_str}' is not a digit in '{spec_path.name}'. Skipping.")
                continue
            clip_idx = int(clip_idx_str)


            if parsed_name_stem_from_npy in filename_to_label_map:
                primary_label_int, original_full_filename = filename_to_label_map[parsed_name_stem_from_npy]

                clip_entry = {
                    'original_filename': original_full_filename,
                    'spec_npy_filename': spec_path.name,
                    'clip_index': clip_idx,
                    'primary_label_int': primary_label_int
                }
                clip_samples.append(clip_entry)
                manifest_data.append(clip_entry)
            else:
                if processed_count < 20: # Print warning only for the first few mismatches to avoid flooding
                    print(f"Warning: No match in train_df lookup for parsed .npy stem '{parsed_name_stem_from_npy}' from file '{spec_path.name}'")

            processed_count += 1

        except Exception as e:
            print(f"Error parsing or mapping {spec_path.name}: {e}")

    cfg.TOTAL_CLIPS = len(clip_samples)
    print(f"Created {cfg.TOTAL_CLIPS} clip samples from precomputed files.")

    if not clip_samples:
         print("CRITICAL ERROR: clip_samples list is empty. Check mapping logic and .npy filenames based on debug output.")
    else:
        manifest_df = pd.DataFrame(manifest_data)
        manifest_save_path = cfg.PRECOMPUTED_SPECS_PATH / "manifest.csv"
        try:
            manifest_df.to_csv(manifest_save_path, index=False)
            print(f"Manifest file saved to: {manifest_save_path}")
            print(manifest_df.head())
        except Exception as e:
            print(f"Error saving manifest to {manifest_save_path}: {e}")
            print("Attempting to save to /content/manifest.csv instead...")
            manifest_save_path_content = Path("/content/manifest.csv")
            manifest_df.to_csv(manifest_save_path_content, index=False)
            print(f"Manifest file saved to: {manifest_save_path_content}")
            print(manifest_df.head())



Train metadata loaded. Shape: (28564, 13)
206 unique classes found.
Found 187904 precomputed .npy files.
Generating clip_info_list and manifest from precomputed files and train_df...
Generated 28564 keys for lookup map.
Sample keys from filename_to_label_map (first 5):
  '1139490_CSA36385.ogg' -> (0, '1139490/CSA36385.ogg')
  '1139490_CSA36389.ogg' -> (0, '1139490/CSA36389.ogg')
  '1192948_CSA36358.ogg' -> (1, '1192948/CSA36358.ogg')
  '1192948_CSA36366.ogg' -> (1, '1192948/CSA36366.ogg')
  '1192948_CSA36373.ogg' -> (1, '1192948/CSA36373.ogg')


Mapping precomputed files:   0%|          | 0/187904 [00:00<?, ?it/s]

  Attempting to match .npy parsed stem: 'strowl1_XC48736.ogg'
  Attempting to match .npy parsed stem: 'roahaw_XC113755.ogg'
  Attempting to match .npy parsed stem: 'bkcdon_XC250791.ogg'
  Attempting to match .npy parsed stem: 'bkmtou1_XC821017.ogg'
  Attempting to match .npy parsed stem: 'yebsee1_XC216248.ogg'
Created 187904 clip samples from precomputed files.
Manifest file saved to: /content/temp_data/extracted/kaggle/working/precomputed_specs_np/manifest.csv
      original_filename                spec_npy_filename  clip_index  \
0   strowl1/XC48736.ogg    strowl1_XC48736.ogg_clip1.npy           1   
1   roahaw/XC113755.ogg    roahaw_XC113755.ogg_clip1.npy           1   
2   bkcdon/XC250791.ogg    bkcdon_XC250791.ogg_clip2.npy           2   
3  bkmtou1/XC821017.ogg  bkmtou1_XC821017.ogg_clip17.npy          17   
4  yebsee1/XC216248.ogg  yebsee1_XC216248.ogg_clip11.npy          11   

   primary_label_int  
0                175  
1                144  
2                 70  
3        

In [79]:
manifest_df

,original_filename,spec_npy_filename,clip_index,primary_label_int
0,strowl1/XC48736.ogg,strowl1_XC48736.ogg_clip1.npy,1,175
1,roahaw/XC113755.ogg,roahaw_XC113755.ogg_clip1.npy,1,144
2,bkcdon/XC250791.ogg,bkcdon_XC250791.ogg_clip2.npy,2,70
3,bkmtou1/XC821017.ogg,bkmtou1_XC821017.ogg_clip17.npy,17,71
4,yebsee1/XC216248.ogg,yebsee1_XC216248.ogg_clip11.npy,11,197
...,...,...,...,...
187899,blbgra1/XC586461.ogg,blbgra1_XC586461.ogg_clip5.npy,5,72
187900,yeofly1/XC263206.ogg,yeofly1_XC263206.ogg_clip2.npy,2,203
187901,whtdov/XC235857.ogg,whtdov_XC235857.ogg_clip22.npy,22,190
187902,bbwduc/iNat883928.ogg,bbwduc_iNat883928.ogg_clip0.npy,0,68


In [75]:
clip_samples = []
manifest_data = [] # To store data for the CSV


filename_to_label_map = {}
for _, row in train_df.iterrows():
    original_filename = row['filename']
    # Create the stem exactly as you did when saving the .npy files
    filename_stem_for_lookup = original_filename.replace('/', '_').replace('.ogg', '')
    filename_to_label_map[filename_stem_for_lookup] = (row['primary_label_int'], original_filename)

for spec_path in tqdm(all_precomputed_files, desc="Mapping precomputed files"):
    try:
        # Parse filename and clip_idx from spec_path.name
        # Assumes format like "PROCESSED_FILENAME_STEM_clipIDX.npy"
        # e.g., "common_XC12345_clip0.npy"
        npy_filename_stem = spec_path.stem # e.g., "common_XC12345_clip0"
        parts = npy_filename_stem.split('_clip')

        if len(parts) != 2:
            print(f"Warning: Could not parse clip index from {spec_path.name}. Skipping.")
            continue

        parsed_name_stem = parts[0] # e.g., "common_XC12345"
        clip_idx = int(parts[1])

        if parsed_name_stem in filename_to_label_map:
            primary_label_int, original_filename = filename_to_label_map[parsed_name_stem]

            clip_entry = {
                'original_filename': original_filename, # Original filename for reference
                'spec_npy_filename': spec_path.name, # Actual .npy filename
                'clip_index': clip_idx,
                'primary_label_int': primary_label_int
            }
            clip_samples.append(clip_entry)
            manifest_data.append(clip_entry) # Add to manifest data as well
        else:
            print(f"Warning: No match in train_df lookup for parsed stem '{parsed_name_stem}' from {spec_path.name}")

    except Exception as e:
        print(f"Error parsing or mapping {spec_path.name}: {e}")

cfg.TOTAL_CLIPS = len(clip_samples)
print(f"Created {cfg.TOTAL_CLIPS} clip samples from precomputed files.")

if not clip_samples:
      print("CRITICAL ERROR: clip_samples list is empty. Check mapping logic and .npy filenames.")
      # Stop execution or handle
else:
    # --- Save the Manifest File ---
    manifest_df = pd.DataFrame(manifest_data)
    # Save it in a location you can easily access or re-upload to Colab
    # For example, in the same directory as precomputed specs or base_data_path

    manifest_save_path = Path("/content/manifest.csv")

    manifest_df.to_csv(manifest_save_path, index=False)
    print(f"Manifest file saved to: {manifest_save_path}")
    print(manifest_df.head())


Mapping precomputed files:   0%|          | 0/187904 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Created 0 clip samples from precomputed files.
CRITICAL ERROR: clip_samples list is empty. Check mapping logic and .npy filenames.
